Note: To display the density grid calculations, in VisIt, load in the .cube file (must select file type and deselect default option), and deselect options to apply options to all plots/windows. When making density grid plots, use the minimum and maximum energy values displayed inthe legend (don't turn off annotations in VisIt immediately) to *normalize* the color gradient. Make a low quality plot and find a good viewing angle; then, increase the resolution and print screen as png. 

TODO: try making density grid plots using alternative software posted in slack.

In [1]:
using PorousMaterials
using JLD2

In [2]:
set_path_to_data(joinpath(pwd(), "../../data"))
# @eval PorousMaterials PATH_TO_CRYSTALS=joinpath(pwd(),"structural_relaxation","post-relaxation_cifs")
# @eval PorousMaterials PATH_TO_CRYSTALS=joinpath(pwd(), "images/grid_images/replicated_xtals/")

general data folder: /home/ng/DTRA/figures/grid_images/../../data
	crystal structures (.cif, .cssr): /home/ng/DTRA/figures/grid_images/../../data/crystals
	force field files (.csv): /home/ng/DTRA/figures/grid_images/../../data/forcefields
	molecule input files: /home/ng/DTRA/figures/grid_images/../../data/molecules
	simulation output files: /home/ng/DTRA/figures/grid_images/../../data/simulations
	grids (.cube): /home/ng/DTRA/figures/grid_images/../../data/grids


In [3]:
###
#  Set flag for desired calculation
###
calc_energy_grid   = false # flag for energy grid calculations
write_density_grid = true  # flag for extracting density grid from *.jld2 sim files 

true

In [4]:
# preset dictionary vlues
params = Dict(:adsorbate   => "Xe",
              :ljff        => "UFF",
              :grid_points => (100, 100, 100), 
              :repfactors  => (1, 1, 1),
              :temperature => 298.0, 
              :n_rotations => 750,
              :energy_min  => Tuple{Float64, CartesianIndex{3}},
              :xyz_file    => false,
              :vtk_file    => false)

Dict{Symbol, Any} with 9 entries:
  :ljff        => "UFF"
  :adsorbate   => "Xe"
  :grid_points => (100, 100, 100)
  :temperature => 298.0
  :energy_min  => Tuple{Float64, CartesianIndex{3}}
  :xyz_file    => false
  :vtk_file    => false
  :repfactors  => (1, 1, 1)
  :n_rotations => 750

In [5]:
# I need to put place the adsorbate at the location of the minimum energy
function adsorbate_in_mof(energy_grid_params::Dict{String,Dict{Symbol,Any}}, xtal::Crystal, grid::Grid)
    filename = energy_grid_params[xtal.name][:adsorbate] * "_in_" * xtal.name
    
    molecule = Molecule(energy_grid_params[xtal.name][:adsorbate])
    xyz = Tuple([energy_grid_params[xtal.name][:energy_min][2][i] for i in 1:3])
    
    xf_minE = id_to_xf(xyz, grid.n_pts) # fractional coords of min
    x_minE  = Cart(Frac(xf_minE), xtal.box) # Cartesian coords of min
    # make struct
    xe_atom_c = Atoms(1, [molecule.species], x_minE) 

    return write_xyz(xe_atom_c, filename)
end

adsorbate_in_mof (generic function with 1 method)

In [7]:
xtal_names = ["NiPyC2_experiment.cif", 
              "NiPyC2_relax_sc211_meta_functionalized_NH2_pbesol_relax.cif",
              "NiPyC2_relax_sc211_meta_functionalized_CH3_pbesol_relax.cif",
              "NiPyC2_relax_sc211_meta_functionalized_N-C-O_pbesol_relax.cif"]

energy_grid_params = Dict(zip(xtal_names, [params for i in 1:length(xtal_names)]))

Dict{String, Dict{Symbol, Any}} with 4 entries:
  "NiPyC2_relax_sc211_meta… => Dict(:ljff=>"UFF", :adsorbate=>"Xe", :grid_point…
  "NiPyC2_experiment.cif"   => Dict(:ljff=>"UFF", :adsorbate=>"Xe", :grid_point…
  "NiPyC2_relax_sc211_meta… => Dict(:ljff=>"UFF", :adsorbate=>"Xe", :grid_point…
  "NiPyC2_relax_sc211_meta… => Dict(:ljff=>"UFF", :adsorbate=>"Xe", :grid_point…

In [8]:
# special case
energy_grid_params["NiPyC2_experiment.cif"][:repfactors] = (2, 2, 2)

(2, 2, 2)

## Density Grid from `.jld2` File

In [11]:
if write_density_grid
    for xtal_name in xtal_names
        # load crystal and replicate 
        crystal = Crystal(xtal_name)
#         crystal = replicate(crystal, energy_grid_params[xtal][:repfactors])
#         strip_numbers_from_atom_labels!(crystal)

        # assign energy calculation parameters
        mol    = Molecule(energy_grid_params[xtal_name][:adsorbate])
        ljff   = LJForceField(energy_grid_params[xtal_name][:ljff])
        temp   = energy_grid_params[xtal_name][:temperature]      
        
        
        pressures = [0.01, 0.1, 1.0]
        nburn     = 50000
        nsample   = 50000

        # loop over pressures
        for p in pressures
            filename = μVT_output_filename(crystal, [mol], temp, [p], ljff, nburn, nsample)
            println(filename)

            @load joinpath(PorousMaterials.PATH_TO_SIMS, filename) results

            grid = results["density grid"]

            # find the value and location of the minimum energy (useful when plotting) 
            energy_grid_params[xtal_name][:energy_min] = findmin(grid.data)
            println(xtal_name, " energy min = ", energy_grid_params[xtal_name][:energy_min])

            # write cube file using crystal name for filename
            write_cube(grid, filename)
        end

#         # write xyz file
#         if energy_grid_params[xtal][:xyz_file]
#             atoms_c = Cart(crystal.atoms, crystal.box)
#             write_xyz(crystal)
#         end
#         # write vtk file
#         if energy_grid_params[xtal][:vtk_file]
#             write_vtk(crystal)
#         end
    end
end

┌ Info: Crystal NiPyC2_experiment.cif has Pn space group. I am converting it to P1 symmetry.
│         To afrain from this, pass `convert_to_p1=false` to the `Crystal` constructor.
└ @ PorousMaterials /home/ng/.julia/dev/PorousMaterials/src/crystal.jl:426


muVT_xtal_NiPyC2_experiment.cif_T_298.000K_Xe_P_0.010000bar_UFF_50000burn_50000sample.jld2
NiPyC2_experiment.cif energy min = (0.0, CartesianIndex(1, 1, 1))
	See /home/ng/DTRA/figures/grid_images/../../data/grids/muVT_xtal_NiPyC2_experiment.cif_T_298.000K_Xe_P_0.010000bar_UFF_50000burn_50000sample.jld2.cube
muVT_xtal_NiPyC2_experiment.cif_T_298.000K_Xe_P_0.100000bar_UFF_50000burn_50000sample.jld2
NiPyC2_experiment.cif energy min = (0.0, CartesianIndex(1, 1, 1))
	See /home/ng/DTRA/figures/grid_images/../../data/grids/muVT_xtal_NiPyC2_experiment.cif_T_298.000K_Xe_P_0.100000bar_UFF_50000burn_50000sample.jld2.cube
muVT_xtal_NiPyC2_experiment.cif_T_298.000K_Xe_P_1.000000bar_UFF_50000burn_50000sample.jld2
NiPyC2_experiment.cif energy min = (0.0, CartesianIndex(1, 1, 1))
	See /home/ng/DTRA/figures/grid_images/../../data/grids/muVT_xtal_NiPyC2_experiment.cif_T_298.000K_Xe_P_1.000000bar_UFF_50000burn_50000sample.jld2.cube
muVT_xtal_NiPyC2_relax_sc211_meta_functionalized_NH2_pbesol_relax.cif_T_2

## Energy Grid Calculations

In [10]:
if calc_energy_grid
    for xtal in xtal_names
        # load crystal and replicat (if needed)
        crystal = replicate(Crystal(xtal), energy_grid_params[xtal][:repfactors])
        strip_numbers_from_atom_labels!(crystal)

        # assign energy calculation parameters
        mol    = Molecule(energy_grid_params[xtal][:adsorbate])
        ljff   = LJForceField(energy_grid_params[xtal][:ljff])
        points = energy_grid_params[xtal][:grid_points]
        temp   = energy_grid_params[xtal][:temperature]
        rot    = energy_grid_params[xtal][:n_rotations]

        # perform energy grid calculation
        grid = energy_grid(crystal, mol, ljff; n_pts=points, temperature=temp, n_rotations=rot)

        # find the value and location of the minimum energy 
        energy_grid_params[xtal][:energy_min] = findmin(grid.data)
        println(xtal, " energy min = ", energy_grid_params[xtal][:energy_min])

        # write cube file using crystal name for filename
        write_cube(grid, "Egrid_" * crystal.name)

        # write xyz file
        if energy_grid_params[xtal][:xyz_file]
            atoms_c = Cart(crystal.atoms, crystal.box)
            write_xyz(crystal)
        end

        # write vtk file
        if energy_grid_params[xtal][:vtk_file]
            write_vtk(crystal)
        end

    #     adsorbate_in_mof(energy_grid_params, crystal, grid)
    end
end